In [1]:
from __future__ import division
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer
import numpy as np
import operator
from gensim.models import Word2Vec
import sys
sys.path.append('/var/sparkamplify/data_process/')
import db
from flashtext import KeywordProcessor
from nltk.stem.snowball import SnowballStemmer 
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
#import nltk
import string
from multiprocessing import cpu_count
import time
from math import sqrt, log
import os
import pickle
from datetime import datetime as dt

In [2]:
from article_tagging_min import *

In [3]:
file_date='{:02d}{:02d}'.format(dt.today().month,dt.today().day)
file_date

'0613'

In [4]:
stemmer = WordNetLemmatizer().lemmatize
stop = stopwords.words('english') + list(string.punctuation)

In [99]:
kwrp=KeywordProcessor(case_sensitive=True)
kwsyn = {'ai':['artificial intelligence'],
         'ar':['augmented reality'],
         'smartglass':['google glass'],
         'biotech':['bioscience','biotech company'],
         'blockchain':['blockchains','blockchain technology'],
         'cryptocurrency':['cryptocurrencies','litecoin'],
         'ethereum':['eth'],
         'medium':['media'],
         'retail':['retailer'],
         'data science':['data scientist'],
         'drone':['drone delivery','delivery drone','drones'],'ecommerce':['e-commerce','e commerce']}
kwrp.add_keywords_from_dict(kwsyn)
kwrp.get_all_keywords()

{'artificial intelligence': 'ai',
 'augmented reality': 'ar',
 'google glass': 'smartglass',
 'bioscience': 'biotech',
 'biotech company': 'biotech',
 'blockchains': 'blockchain',
 'blockchain technology': 'blockchain',
 'cryptocurrencies': 'cryptocurrency',
 'litecoin': 'cryptocurrency',
 'eth': 'ethereum',
 'e-commerce': 'ecommerce',
 'e commerce': 'ecommerce',
 'media': 'medium',
 'retailer': 'retail',
 'data scientist': 'data science',
 'drone delivery': 'drone',
 'drones': 'drone',
 'delivery drone': 'drone'}

In [100]:
pickle.dump(kwrp, open('../category_reference/keyword_replacer.pickle','wb'))

In [7]:
def dictsort(dicty,reverse=False,value=True):
    if value:
        return sorted(dicty.items(), key=operator.itemgetter(1),reverse=reverse)
    else:
        return [i[0] for i in sorted(dicty.items(), key=operator.itemgetter(1),reverse=reverse)]

In [10]:
print("Stop word building ...")
stop_words_list = stopword_building()

#General Processing Building
print("Bigrams building ...")
bigram_list = bigram_proc()  

Stop word building ...
making stopword_list
Bigrams building ...
making bigram_list


In [11]:
w2v= Word2Vec.load('../W2V/W2V1124_Skip_mc5_w5_77m.bin')
wv=w2v.wv

In [12]:
voc_list=pickle.load(open('/var/sparkamplify/data_process/category_reference/keyword_extract_model/keyword_ind_dict_0507.pickle','rb'))

In [13]:
voc_list=list(voc_list.values())

In [14]:
ar0=pd.read_csv('./processed_text.csv')

In [15]:
ar0.head()

,id,title,text,kw
0,1,text text text text rob reiner say archie bun...,all in the family cast member rob reiner what...,NaN
1,2,text text text text the next time your right ...,"with thanksgiving fast approaching, no doubt ...",NaN
2,3,text text text text trump revives creepy gun ...,a month after suggesting hillary clinton coul...,NaN
3,4,"epson iprint add document, web printing option",use comma to separate multiple email address ...,NaN
4,5,text text text text oregon rancher who prompt...,"steven and dwight hammond, the oregon rancher...",NaN


In [16]:
last_id=ar0.iloc[-1].id

In [17]:
sql="""SELECT id,title,text,keyword
From data_pool.articles 
WHERE (text IS NOT NULL or title IS NOT NULL)and (id>{})
""".format(last_id)
df=db.query(sql)
ar=pd.DataFrame(data=df,columns=['id','title','text','kw'])

In [18]:
ar=ar.replace(' nan',np.nan)

In [19]:
def text_keyword_train(train_text,vocabulary =None):
    td=TfidfVectorizer(stop_words=stop_words_list, min_df=3, vocabulary = vocabulary, 
                       ngram_range=(1,2),sublinear_tf=True)
    tdm=td.fit_transform(train_text)
    ind_vocab={v:k for k,v in td.vocabulary_.items()}
    return td, tdm.tocoo(), ind_vocab

def text_keyword_transform(target_text_list, td_model, fq_model):
    return len(target_text_list), td_model.transform(target_text_list).tocoo(), fq_model.transform(target_text_list).tocoo()

def text_keywords(tdm_coo, fqm_coo, row_len, ind_vocab, topn=5):
    kw_list=[]
    for row in range(row_len):
        if row not in tdm_coo.row:
            print('no text')
            kw_list.append('')
        else:
            fq_data=fqm_coo.data[np.where(fqm_coo.row==row)]
            fq_col=fqm_coo.col[np.where(fqm_coo.row==row)]
            kw_dict={}
            for i in zip(tdm_coo.col[np.where(tdm_coo.row==row)],tdm_coo.data[np.where(tdm_coo.row==row)]):
                fq=fq_data[np.where(fq_col==i[0])]
                if fq>0: #give higher weight to word appears in title
                    fq=fq[0]
                    fqw = fq+1
                    print(ind_vocab[i[0]],'freq: ',fqw)
                else:
                    fqw = 1
                    fq = 0
                if ind_vocab[i[0]].find(' ')>0: #bigram
                    kw_dict[ind_vocab[i[0]]]=((1+i[1]) * 1)*(fqw)
                    print(i[1])
                else:
                    kw_dict[ind_vocab[i[0]]]=((1+i[1]) * 2.5)*(fqw)
                    print(i[1])

            if len(kw_dict)<topn:
                print(dictsort(kw_dict, reverse=True, value=False))
                kws=','.join(dictsort(kw_dict, reverse=True, value=False))
            else:
                print(dictsort(kw_dict, reverse=True, value=False)[:topn])
                kws=','.join(dictsort(kw_dict, reverse=True, value=False)[:topn])
            kw_list.append(kws)
    return kw_list

In [20]:
def countmetrix(corpus,stop_words=stop_words_list, min_df=10,vocabulary=None):
    cectorizer=CountVectorizer(stop_words=stop_words, min_df=min_df,vocabulary=vocabulary,ngram_range=(1,2),
                               token_pattern='(?u)[^(,\s)]\w*\s*\w*[^(,\s)]')
    y_coo=cectorizer.fit_transform(corpus).tocoo()
    vocab=cectorizer.vocabulary_
    
    voc_reverse = {v:k for k,v in vocab.items()}
    voc_count = {}
    total_count=np.unique(y_coo.col,return_counts=True)
    for voc_ind,count in zip(total_count[0],total_count[1]):
        voc_count[voc_reverse[voc_ind]]=count
        
    return y_coo, vocab, voc_count

In [21]:
def freq_dict(corpus):
    CV=CountVectorizer(min_df=1,ngram_range=(1,2),token_pattern='(?u)[^(,\s)]\w*\s*\w*[^(,\s)]')
        
    voc_freq=CV.fit_transform(corpus).tocoo()
    vocab=CV.vocabulary_
    voc_reverse = {v:k for k,v in vocab.items()}

    total_count=np.unique(voc_freq.col,return_counts=True)
    for voc_ind,count in zip(total_count[0],total_count[1]):
        voc_count[voc_reverse[voc_ind]]=count
    return voc_count

In [22]:
def cat_keyword(y_coo, vocab, voc_count, pos=None, neg=None, how_pos='union', 
                topn=20, min_df=10, reverse=False, rank = False):
    # pos = postive keyword list
    # how_pos ={'union', 'intersection'}
    # neg = negative keyword list
    
    if (not pos):
        print('please give postive kw')
    else:
        if pos: #add bigram process
            ind=0
            for cat in pos:
                cat = stemmer(cat)
                if cat in vocab:
                    if ind==0:
                        articles = y_coo.row[np.where(y_coo.col==vocab[cat])]
                        ind+=1
                    else:
                        if how_pos=='union':
                            articles = y_coo.row[np.where((y_coo.col==vocab[cat])|(np.isin(y_coo.row,articles)))]
                        elif how_pos=='intersection':
                            articles = y_coo.row[np.where((y_coo.col==vocab[cat])&(np.isin(y_coo.row,articles)))]
                        else:
                            print('pls choose how to select from positive ketwords /nhow_pos =\{\'union\', \'intersection\'\}')
                else:
                    print(cat+' is not in dict')
            if ind == 0:
                return 'no valid postive keywords'
                 
        if neg:
            for ncat in neg:
                ncat = stemmer(ncat)
                neg_articles =y_coo.row[np.where(y_coo.col==vocab[ncat])]
                articles = articles[np.where(np.isin(articles,neg_articles,invert=True))]
                
        kw_collect = y_coo.col[np.where(np.isin(y_coo.row,articles))]
                                      
        kw_count=np.unique(kw_collect,return_counts=True)
        voc_reverse = {v:k for k,v in vocab.items()}
        kw_prob={}
        for voc_ind, count in zip(kw_count[0],kw_count[1]):
            if not voc_reverse[voc_ind].isdigit():
                if (voc_count[voc_reverse[voc_ind]]>=min_df) and (count>2) \
                and (log(count/(voc_count[voc_reverse[voc_ind]])+1)*sqrt(count)>0.05) :
                    if reverse:
                        kw_prob[voc_reverse[voc_ind]]=(log(count/len(np.unique(articles))+1)*sqrt(count),count,
                                                   voc_count[voc_reverse[voc_ind]]) 
                        
                    else:
                        kw_prob[voc_reverse[voc_ind]]=(log(count/(voc_count[voc_reverse[voc_ind]])+1)*sqrt(count),count,
                                                       voc_count[voc_reverse[voc_ind]]) 
                                      
        if rank:
            return dictsort(kw_prob,reverse=True)[:topn]
        else:
            return kw_prob

In [23]:
def title_proc(title,kwrp=kwrp):
    return ','.join([stemmer(i) for i in word_tokenize(kwrp.replace_keywords(re.sub('[-/\']+','',title.lower()))) 
                     if (i not in stop)])

In [24]:
start=time.time()
ar.loc[ar.text.notnull(),'text']=ar.loc[ar.text.notnull(),'text'].apply(article_lemmatizer_proccess)
print(time.time() - start)

0.0027740001678466797


In [25]:
start=time.time()
ar.loc[ar.title.notnull(),'title']=ar.loc[ar.title.notnull(),'title'].apply(article_lemmatizer_proccess)
print(time.time() - start)

0.002187013626098633


In [26]:
ar=ar0.append(ar)

In [27]:
ar.head()

,id,title,text,kw
0,1,text text text text rob reiner say archie bun...,all in the family cast member rob reiner what...,NaN
1,2,text text text text the next time your right ...,"with thanksgiving fast approaching, no doubt ...",NaN
2,3,text text text text trump revives creepy gun ...,a month after suggesting hillary clinton coul...,NaN
3,4,"epson iprint add document, web printing option",use comma to separate multiple email address ...,NaN
4,5,text text text text oregon rancher who prompt...,"steven and dwight hammond, the oregon rancher...",NaN


In [28]:
textlist=(ar.title+' '+ar.text).fillna('').tolist()

In [29]:
titlelist=ar.title.fillna('').tolist()

In [30]:
len(titlelist)==len(textlist)

True

In [31]:
voc_test=TfidfVectorizer(stop_words=stop_words_list, min_df=3, ngram_range=(1,1))

In [32]:
start=time.time()
z=voc_test.fit(textlist)
print(time.time() - start)

266.01336193084717


In [33]:
singram=list(voc_test.vocabulary_.keys())

In [34]:
voc_list=singram+list(bigram_list)

In [35]:
start=time.time()
model, matrix, voc_dict =text_keyword_train(textlist,vocabulary=voc_list)
print(time.time() - start)

/var/sparkamplify/data_process/article_tagging_model/.venv/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


436.75917768478394


In [36]:
pickle.dump(model, open('../category_reference/keyword_extract_model/keyword_extract_{}.pickle'.format(file_date),'wb'))

In [37]:
pickle.dump(voc_dict, open('../category_reference/keyword_extract_model/keyword_ind_dict_{}.pickle'.format(file_date),'wb'))

In [38]:
fq_dict={v:k for k,v in voc_dict.items()}

In [39]:
fq=CountVectorizer(stop_words=stop_words_list,vocabulary=fq_dict,ngram_range=(1,2))

In [40]:
fqm=fq.fit_transform(textlist).tocoo()

In [41]:
fqtitle = fq.transform(titlelist).tocoo()

In [42]:
fq.vocabulary_ == model.vocabulary_

True

In [43]:
pickle.dump(fq, open('../category_reference/keyword_extract_model/keyword_freq_{}.pickle'.format(file_date),'wb'))

In [44]:
ar.to_csv('processed_text.csv',index=None)

In [45]:
start=time.time()
ar.loc[ar.title.notnull(),'title']=ar.loc[ar.title.notnull(),'title'].apply(lambda x: title_proc(x))
print(time.time() - start)

157.3902609348297


In [46]:
kwinput=(ar.title.fillna('')+','+ar.kw.fillna('')).apply(lambda x: ','.join(set(x.split(',')))).tolist()

In [47]:
start=time.time()
ca, voc, voc_count=countmetrix(kwinput,stop_words=stop,vocabulary=voc_list)
print(time.time() - start)

20.794970750808716


In [48]:
pickle.dump(ca, open('../category_reference/relative_word_matrix/count_matrix_coo_{}.pickle'.format(file_date),'wb'))
pickle.dump(voc, open('../category_reference/relative_word_matrix/count_matrix_vocab_dict_{}.pickle'.format(file_date),'wb'))
pickle.dump(voc_count, open('../category_reference/relative_word_matrix/count_matrix_freq_dict_{}.pickle'.format(file_date),'wb'))

In [49]:
tet =['airbus', 'ditch', 'microsoft,', 'fly', 'off', 'to', 'google']

In [50]:
nltk.pos_tag(tet)

[('airbus', 'JJ'),
 ('ditch', 'NN'),
 ('microsoft,', 'NN'),
 ('fly', 'VBD'),
 ('off', 'RB'),
 ('to', 'TO'),
 ('google', 'VB')]

In [51]:
tet=titlelist[26677]

In [52]:
def title_proc0(title):
    return [stemmer(i) for i in word_tokenize(kwrp.replace_keywords(re.sub('[-/\']+','',title.lower()))) if (i not in stop)]

In [53]:
nltk.pos_tag(title_proc0(tet))

[('could', 'MD'),
 ('next', 'RB'),
 ('united', 'JJ'),
 ('nation', 'NN'),
 ('secretary', 'NN'),
 ('general', 'JJ'),
 ('woman', 'NN')]

In [54]:
zz = pickle.load(open('../category_reference/category_kw/top20_sim_words0320_2018.pickle','rb'))
cats = list(zz.keys())
cats+=['other','ar/vr']

In [55]:
cats=cats[2:]

In [56]:
bigram_set=[v for v in voc_list if ' ' in v]

In [57]:
cat_keyword(ca, voc, voc_count, pos=['usb 3.5'],neg=[],topn=50,min_df=10,how_pos='union',rank=True)

usb 3.5 is not in dict


'no valid postive keywords'

In [58]:
sim_kw={}
for cat in cats:
    
    print(cat)
    sim_kw[cat]={}
    if cat == 'other':
        pass
    
    elif cat == 'ar/vr':
        cat_family=['ar','vr','mixed reality']
        for v in bigram_set:
            if 'ar' in v.split(' '):
                cat_family.append(v)
            if 'vr' in v.split(' '):
                cat_family.append(v)
        print(cat_family)
        sim_kw[cat]= cat_keyword(ca, voc, voc_count, pos=cat_family,
                                 neg=[],topn=50,min_df=10,how_pos='union')

    elif cat == 'gaming':
        cat_family=['console','esports','videogame']
        for v in bigram_set:
            if 'game' in v.split(' '):
                cat_family.append(v)
            if 'gaming' in v.split(' '):
                cat_family.append(v)
        print(cat_family)
        sim_kw[cat]= cat_keyword(ca, voc, voc_count, pos=cat_family,
                                 neg=['throne','sport','sox game'],topn=50,min_df=10,how_pos='union')
           
    elif cat == 'media':
        cat_family=['medium']
        for v in bigram_set:
            if 'medium' in v.split(' '):
                cat_family.append(v)
        print(cat_family)
        sim_kw[cat]= cat_keyword(ca, voc, voc_count, pos=cat_family,neg=[],topn=50,min_df=10,how_pos='union')
           
    elif cat == 'biotech':
        cat_family=[cat]
        for v in bigram_set:
            if cat in v.split(' '):
                cat_family.append(v)
        print(cat_family)
        sim_kw[cat]=cat_keyword(ca, voc, voc_count, pos=cat_family+['biopharma','gene','genetics'],neg=[],topn=50,min_df=10,how_pos='union')
        
    elif cat == 'mobility':
        cat_family=[cat]
        for v in bigram_set:
            if 'mobile' in v.split(' '):
                cat_family.append(v)
        print(cat_family)
        sim_kw[cat]= cat_keyword(ca, voc, voc_count, pos=cat_family+['mobile'],neg=[],topn=50,min_df=10,how_pos='union')
    elif cat == 'health':
        cat_family=[cat]
        for v in bigram_set:
            if cat in v.split(' '):
                cat_family.append(v)
        print(cat_family) #mental #diet
        sim_kw[cat]= cat_keyword(ca, voc, voc_count, pos=cat_family+['depression','diet'],neg=[],topn=50,min_df=10,how_pos='union')
           
    elif cat == 'ai':
        cat_family=[cat,'artificial intelligence']
        for v in bigram_set:
            if cat in v.split(' '):
                cat_family.append(v)
        print(cat_family)
        sim_kw[cat]= cat_keyword(ca, voc, voc_count, pos=cat_family,neg=['weiwei'],topn=50,min_df=10,how_pos='union')
        
    elif cat == 'robotic':
        cat_family=[cat,'robot','humanoid','robotics']
        for v in bigram_set:
            if 'robot' in v.split(' '):
                cat_family.append(v)
        print(cat_family)
        sim_kw[cat]= cat_keyword(ca, voc, voc_count, pos=cat_family,neg=[],topn=50,min_df=10,how_pos='union')
        
    elif cat == 'design':
        cat_family=[cat,'designer']
        for v in bigram_set:
            if cat in v.split(' '):
                cat_family.append(v)
        print(cat_family)
        sim_kw[cat]= cat_keyword(ca, voc, voc_count, pos=cat_family,neg=[],topn=50,min_df=10,how_pos='union')
    
    elif cat == 'advertising':
        cat_family=[cat,'advertisement','ad']
        for v in bigram_set:
            if cat in v.split(' '):
                cat_family.append(v)
            if 'ad' in v.split(' '):
                cat_family.append(v)
        print(cat_family)
        sim_kw[cat]= cat_keyword(ca, voc, voc_count, pos=cat_family,neg=[],topn=50,min_df=10,how_pos='union')
    
    elif cat == 'fintech':
        cat_family=[cat,'mobile payment','cryptocurrency','mobile wallet','banking app']
        for v in bigram_set:
            if cat in v.split(' '):
                cat_family.append(v)
        print(cat_family)
        sim_kw[cat]= cat_keyword(ca, voc, voc_count, pos=cat_family,neg=[],topn=50,min_df=10,how_pos='union')
        
    elif cat == 'ecommerce':
        cat_family=[cat,'online store','online sale','shopping experience','online retailer',
                    'online shopping','online retail']
        for v in bigram_set:
            if cat in v.split(' '):
                cat_family.append(v)
        print(cat_family)
        sim_kw[cat]= cat_keyword(ca, voc, voc_count, pos=cat_family,neg=[],topn=50,min_df=10,how_pos='union')
           
    else:
        cat_family=[cat]
        for v in bigram_set:
            if cat in v.split(' '):
                cat_family.append(v)
        print(cat_family)
        sim_kw[cat]= cat_keyword(ca, voc, voc_count, pos=cat_family,neg=[],topn=50,min_df=10,how_pos='union')

ai
['ai', 'artificial intelligence', 'google ai', 'voice ai', 'ai helping', 'intelligence ai', 'ai assistant', 'ai technology', 'ai machine', 'ai lab', 'ai startup']
blockchain
['blockchain', 'beat blockchain', 'blockchain technology']
health
['health', 'health care', 'digital health', 'health insurance', 'mental health', 'public health', 'health record', 'health data', 'health system', 'health plan', 'health benefit', 'electronic health', 'health service', 'health company', 'reproductive health', 'health issue', 'health tool', 'behavioral health', 'personal health', 'health wellness', 'woman health', 'connected health', 'deepmind health']
biotech
['biotech', 'biotech company']
ecommerce
['ecommerce', 'online store', 'online sale', 'shopping experience', 'online retailer', 'online shopping', 'online retail', 'ecommerce company', 'ecommerce giant', 'ecommerce site', 'ecommerce platform']
enterprise
['enterprise', 'enterprise software', 'enterprise iot', 'packard enterprise', 'enterprise

In [59]:
pickle.dump(sim_kw,open('../category_reference/category_kw/dict_relate_words_{}_2018.pickle'.format(file_date),'wb'))

In [60]:
sim_kw.keys()

dict_keys(['ai', 'blockchain', 'health', 'biotech', 'ecommerce', 'enterprise', 'fintech', 'media', 'advertising', 'gaming', 'robotic', 'drone', 'mobility', 'security', 'design', 'other', 'ar/vr'])

In [61]:
file_date

'0613'

In [62]:
sim_kw=pickle.load(open('../category_reference/category_kw/dict_relate_words_{}_2018.pickle'.format('0613'),'rb'))

In [63]:
hdd=cat_keyword(ca, voc, voc_count, pos=[
    'hardware','cpu','gpu','core','processor','pc','laptop','battery','hard drive',
    'ram','chip','microchip','motherboard'
],neg=[],topn=50,min_df=10,how_pos='union')

In [64]:
iott=cat_keyword(ca, voc, voc_count, pos=['iot'],neg=[],topn=50,min_df=10,how_pos='union')

In [65]:
dictsort(hdd,reverse=True)

[('pc', (29.837550750529562, 1853, 1853)),
 ('battery', (27.15686609983632, 1535, 1535)),
 ('chip', (23.42386265957402, 1142, 1142)),
 ('laptop', (23.228188363067407, 1123, 1123)),
 ('core', (19.506890757453263, 792, 792)),
 ('hardware', (18.456467345296737, 709, 709)),
 ('ram', (14.85018294124535, 459, 459)),
 ('processor', (13.828242821828237, 398, 398)),
 ('cpu', (12.553429354768763, 328, 328)),
 ('gpu', (10.139869080934679, 214, 214)),
 ('motherboard', (9.753445378726632, 198, 198)),
 ('hard drive', (9.728784288999611, 197, 197)),
 ('intel', (8.089027477498782, 581, 1457)),
 ('battery life', (6.985467273346363, 333, 714)),
 ('gamer', (6.192846618250607, 200, 364)),
 ('i7', (5.642082945001784, 80, 91)),
 ('amd', (5.50034462564519, 256, 624)),
 ('i9', (5.123170613886321, 60, 64)),
 ('gaming', (4.891916594317653, 449, 1729)),
 ('ryzen', (4.52344434586152, 109, 201)),
 ('microchip', (4.2162496978918895, 37, 37)),
 ('xps', (4.1256416466714665, 78, 131)),
 ('spectre', (3.8688615876937256

In [105]:
bg=cat_keyword(ca, voc, voc_count, pos=['japan'],neg=[],topn=50,min_df=10,how_pos='union')

In [106]:
dictsort(bg,reverse=True)

[('japan', (38.22378337272817, 3041, 3041)),
 ('js', (21.971068992469498, 1130, 1225)),
 ('elementsbytagname', (21.84318642332453, 1035, 1065)),
 ('node', (21.578622184791783, 1088, 1178)),
 ('techcrunch', (21.53279082080628, 1189, 1371)),
 ('elementbyid', (20.961609618104177, 955, 984)),
 ('http', (20.140008903483835, 1084, 1285)),
 ('covering', (3.007541540191337, 100, 285)),
 ('japn', (2.710721720759371, 20, 24)),
 ('tokyo', (2.5938137569694004, 101, 343)),
 ('abe', (2.5267914823999758, 38, 75)),
 ('jetro', (1.8514042952798477, 37, 104)),
 ('jepang', (1.7594284489776184, 27, 67)),
 ('kyoto', (1.682141667037733, 22, 51)),
 ('okinawa', (1.4893106589416731, 9, 14)),
 ('osaka', (1.4738548802978406, 14, 29)),
 ('fukushima', (1.2205316655387233, 18, 54)),
 ('asia', (1.1887584176789916, 130, 1183)),
 ('abenomics', (1.1512690689082972, 6, 10)),
 ('japanese', (1.0557861407866582, 84, 688)),
 ('shinzo', (1.0400827425477763, 8, 18)),
 ('nhk', (0.9295559358588303, 6, 13)),
 ('abes', (0.87367161